In [ ]:
import pandas as pd
import time
import pygsheets
from datetime import datetime, timedelta
import ast

In [ ]:
# Write to G-Sheets
gc = pygsheets.authorize(service_file = '/Users/mdavis/dev/ironman/ironmanmaps-cce634a56b5e.json')

In [ ]:
wb = gc.open('Triathlon Results')

In [ ]:
wks = wb.worksheet('title','AltNutr')

In [ ]:
df = wks.get_as_df()

In [ ]:
Nutrition = df[df['Fluid (mL)'] != '']
Nutrition

In [ ]:
params = df[ (df['What'] != '') & (df['Concentration'] != '') & (df['Fluid (mL)'] == '')][['What', 'Concentration']]
params.set_index('What', inplace = True)
params = params.to_dict(orient = 'dict')['Concentration']
params

In [ ]:
# Create dictionary with times
time_dict = {}

# Convert aid stations to time
swim_distace = 2112
bike_distance = 56
run_distance = 13.1

# Swim Start
swim_start = pd.to_datetime('20220911 ' + params['Swim Start Time'], format = '%Y%m%d %H:%M %p')
time_dict['Swim Start'] = swim_start

# Pre-Swim Nutrition (done after swim start - just to not have to redo logic)
time_dict['Pre-Swim Nutrition Start'] = swim_start - timedelta(minutes = 25)

# Swim Duration
swim_pace = datetime.strptime(params['Swim Average Speed'].split(' ')[0], "%M:%S")
swim_delta = timedelta(minutes = swim_pace.minute, seconds = swim_pace.second)
swim_duration = (swim_distace / 100) * swim_delta
t1_start = swim_start + swim_duration
time_dict['T1 Start'] = t1_start

# T1
t1_time = datetime.strptime(params['T1'], "%M:%S")
t1_duration = timedelta(minutes = t1_time.minute, seconds = t1_time.second)
bike_start = t1_start + t1_duration
time_dict['Bike Start'] = bike_start

# Bike
bike_pace = float(params['Bike Average Speed'].split(' ')[0])
bike_duration = timedelta(hours = bike_distance / bike_pace)
t2_start = bike_start + bike_duration
time_dict['T2 Start'] = t2_start

# T2
t2_time = datetime.strptime(params['T2'], "%M:%S")
t2_duration = timedelta(minutes = t2_time.minute, seconds = t2_time.second)
run_start = t2_start + t2_duration
time_dict['Run Start'] = run_start

# Run
run_pace = datetime.strptime(params['Run Average Speed'].split(' ')[0], "%M:%S")
run_delta = timedelta(minutes = run_pace.minute, seconds = run_pace.second)
run_duration = run_distance * run_delta
finish = run_start + run_duration
time_dict['Finish'] = finish

# Bike Aid Stations
bike_aid_stations = ast.literal_eval(params['Bike Aid Stations'])
bike_aid_station_times = list()
for bike_aid_station in bike_aid_stations:
    bike_aid_station_time = bike_start + timedelta(hours = bike_aid_station / bike_pace)
    time_dict['Bike Aid Station @ Mile ' + str(bike_aid_station)] = bike_aid_station_time

# Run Aid Stations
run_aid_stations = ast.literal_eval(params['Run Aid Stations'])
run_aid_station_times = list()
for run_aid_station in run_aid_stations:
    run_aid_station_time = run_start + (run_aid_station * run_delta)
    time_dict['Run Aid Station @ Mile ' + str(run_aid_station)] = run_aid_station_time

# Create this as a dataframe
milestone_df = pd.DataFrame.from_dict(time_dict, orient = 'index', columns = ['Time'])
milestone_df.reset_index(drop = False, inplace = True)
milestone_df.rename(columns = {'index': 'Milestone'}, inplace = True)
milestone_df = milestone_df[['Time', 'Milestone']]
milestone_df.sort_values(by = ['Time', 'Milestone'], ascending = [True, True], inplace = True)
milestone_df.reset_index(drop = True, inplace = True)

# Add the event column
milestone_df.loc[:, 'Event'] = 'Swim'
milestone_df.loc[milestone_df['Time'] >= t1_start, 'Event'] = 'T1'
milestone_df.loc[milestone_df['Time'] >= bike_start, 'Event'] = 'Bike'
milestone_df.loc[milestone_df['Time'] >= t2_start, 'Event'] = 'T2'
milestone_df.loc[milestone_df['Time'] >= run_start, 'Event'] = 'Run'
milestone_df.loc[milestone_df['Time'] >= finish, 'Event'] = 'Finish'

# Round to a minute
milestone_df['Time'] = milestone_df['Time'].dt.round('min')
milestone_df

In [ ]:
time_df = milestone_df.copy()

# Add in sweat rate, carb utilization, Sodium Concentration
swim_sweat_rate = params['Swim Sweat Rate']
bike_sweat_rate = params['Bike Sweat Rate']
run_sweat_rate = params['Run Sweat Rate']
carb_expenditure = params['Carb Utilization Rate']
sodium_concentration = params['Sodium Concentration']

time_df.loc[time_df['Event'] == 'Swim', 'SweatRate'] = swim_sweat_rate
time_df.loc[time_df['Event'] == 'T1', 'SweatRate'] = swim_sweat_rate
time_df.loc[time_df['Event'] == 'Bike', 'SweatRate'] = bike_sweat_rate
time_df.loc[time_df['Event'] == 'T2', 'SweatRate'] = bike_sweat_rate
time_df.loc[time_df['Event'] == 'Run', 'SweatRate'] = run_sweat_rate
time_df.loc[time_df['Event'] == 'Finish', 'SweatRate'] = run_sweat_rate

time_df.loc[:, 'SodiumConcentration'] = sodium_concentration
time_df.loc[:, 'CarbUtilization'] = carb_expenditure

# Expand the dataframe per minute
time_df.set_index(time_df['Time'], inplace = True)
time_df = time_df.resample('T').ffill()
time_df.loc[time_df['Time'] != time_df.index, 'Milestone'] = ''
time_df.drop(columns = ['Time'], axis = 1, inplace = True)

# Convert the rates to per minute value
time_df.loc[:, 'FluidExpenditure (mL)'] = time_df['SweatRate'].str.split(' ').str[0].astype(float)/60
time_df.loc[:, 'SodiumExpenditure (mg)'] = time_df['SodiumConcentration'].str.split(' ').str[0].astype(float) * time_df['FluidExpenditure (mL)'] / 1000
time_df.loc[:, 'CarbExpenditure (g)'] = time_df['CarbUtilization'].str.split(' ').str[0].astype(float)/60

time_df

In [ ]:
# Export for testing

wks = wb.worksheet('title','NutritionTimeline')
wks.clear()
wks.set_dataframe(time_df,(1,1))
wks.adjust_column_width(1, len(time_df.columns))

In [ ]:
priors = Nutrition[Nutrition['Frequency/Interval'].str.contains('prior')].copy()
priors.loc[:, 'TimeBefore'] = priors['Frequency/Interval'].str.split(' ').str[0].astype(int)
priors.loc[:, 'Time'] = swim_start - pd.to_timedelta(priors['TimeBefore'], unit = 'T')
priors


In [ ]:
bike = Nutrition[(Nutrition['Frequency/Interval'].str.contains('bike')) & (~Nutrition['Frequency/Interval'].str.contains('prior'))].copy()
bike.loc[:, '']